# Test Gemini vs Ensemble for MMLU


to do:

-ignore claude, add gemma, palm

In [1]:
import os
import copy
import json
import random
import pandas as pd
from pprint import pprint
from datetime import datetime
from datasets import load_dataset

from utils import Annotate
from config import PALM_CONFIG, GEMINI_CONFIG, CLAUDE_CONFIG

In [2]:
seed = 420
now = datetime.now().strftime("%Y%m%d")

In [3]:
dataset = load_dataset("cais/mmlu", "all")
n_sample = 2000
# # take a small sample for dev purposes
dataset = dataset['test'].shuffle(seed=seed).select(range(n_sample))


In [4]:
dataset["question"][dataset['subject'].index("abstract_algebra")]

'Statement 1 | If G, H and K are groups of order 4, at least two of them are isomorphic. Statement 2 | If Z_r + Z_s is isomorphic to a subgroup Z_m + Z_n, then r divides m and s divides n.'

In [5]:
gemini_prompt_template = """
<QUESTION>
{datapoint}
</QUESTION>
------------

<CHOICES>
{labels}
</choices>
------------

INSTRUCTION:
- read the above question carefully.
- you are given 4 choices seperated by comma in <CHOICES>.
- take your time and pick the precise correct answer from <CHOICES> for the given <QUESTION>.
- remember that there is always only one correct answer.
- return the exact correct answer from <CHOICES>. Don't provide explanations.
"""

In [6]:
prompt = [gemini_prompt_template.format(datapoint=x['question'],
                                        labels=x['choices']) for x in dataset]
print(len(prompt))

100


In [7]:
models = [
    "gemini",
    "palm",
    # "claude"
    ]

PALM_CONFIG["project_config"]["qpm"] = 150

palm_1 =  copy.deepcopy(PALM_CONFIG)
palm_1['config_name'] = "temp_0.4"

palm_2 =  copy.deepcopy(PALM_CONFIG)
palm_2['config_name'] = "temp_0.9"
palm_2["generation_config"]['temperature'] = 0.9

GEMINI_CONFIG["project_config"]["qpm"] = 100

gemini_1 =  copy.deepcopy(GEMINI_CONFIG)
gemini_1['config_name'] = "-1.0-pro-001"

gemini_2 =  copy.deepcopy(GEMINI_CONFIG)
gemini_2['config_name'] = "-1.5-flash-001"
gemini_2['"model"'] = "gemini-1.5-flash-001"

gemini_3 =  copy.deepcopy(GEMINI_CONFIG)
gemini_3['config_name'] = "-1.0-ultra-001"
gemini_3['"model"'] = "gemini-1.0-ultra-001"



model_config = {
    "gemini": [
        gemini_1,
        gemini_2,
        gemini_3
         ],
    "palm": [
        palm_1, 
    #     # palm_2
        ],
    # "claude": [
    #     CLAUDE_CONFIG
    # ]
}

ann = Annotate()


In [8]:
output_dict = await ann.classification(prompt, models, model_config)

Gathering gemini_-1.0-pro-001 results:   1%|          | 1/100 [00:57<1:34:04, 57.02s/it]2024-08-13 19:45:01,317/Annotate[ERROR]: gemini_-1.0-pro-001 Task 1 failed: cannot import name 'GenerativeModel' from 'vertexai.generative_models' (unknown location)
2024-08-13 19:45:01,318/Annotate[ERROR]: gemini_-1.0-pro-001 Task 2 failed: cannot import name 'GenerativeModel' from 'vertexai.generative_models' (unknown location)
2024-08-13 19:45:01,318/Annotate[ERROR]: gemini_-1.0-pro-001 Task 3 failed: cannot import name 'GenerativeModel' from 'vertexai.generative_models' (unknown location)
2024-08-13 19:45:01,318/Annotate[ERROR]: gemini_-1.0-pro-001 Task 4 failed: cannot import name 'GenerativeModel' from 'vertexai.generative_models' (unknown location)
2024-08-13 19:45:01,319/Annotate[ERROR]: gemini_-1.0-pro-001 Task 5 failed: cannot import name 'GenerativeModel' from 'vertexai.generative_models' (unknown location)
2024-08-13 19:45:01,320/Annotate[ERROR]: gemini_-1.0-pro-001 Task 6 failed: cannot

In [9]:
llm_response = {}

for k in output_dict.keys():
    llm_response[k] = []
    for idx, r in enumerate(output_dict[k]):
        if r is not None:
            stripped_r = r.strip().strip("'")
            if stripped_r in dataset['choices'][idx]:
                llm_response[k].append(dataset['choices'][idx].index(stripped_r))
            else:
                # Handle case where stripped_r is not found in choices
                llm_response[k].append(None)
        else:
            # Handle None values appropriately
            llm_response[k].append(None)

In [ ]:
# with open(f"./data/{now}/llm_response_{n_sample}__{now}.json", "r") as f:
#     llm_response= json.load(f)

In [11]:
def drop_nones(data):
    df = pd.DataFrame.from_dict(llm_response).dropna().astype(int)
    print(df.shape)
    data = df.to_dict(orient="list")

    return data, df.index.to_list()

def convert_dict_to_indexed_list(data_dict):
    number_map = {key: index for index, key in enumerate(data_dict.keys())}
    max_len = len(next(iter(data_dict.values())))

    result = []
    for index in range(max_len):
        for key, value_list in data_dict.items():
            value = value_list[index]
            converted_value = value 
            result.append([index, number_map[key], converted_value])
    return result
    

def generate_task_config(response_dict, num_classes):

    num_labels = sum(len(lst) for lst in response_dict.values())
    num_tasks =  len(list(response_dict.values())[0])
    num_labelers = len(response_dict)
    z  = 1/num_classes


    tc = [num_labels, num_labelers, num_tasks, num_classes]
    tc.extend([z] * tc[-1])

    return tc

In [12]:
n_class = 4
llm_response, keep_idx = drop_nones(llm_response)
task_conf = generate_task_config(llm_response, n_class)
llm_result_list = convert_dict_to_indexed_list(llm_response)
llm_result_list.insert(0, task_conf)

(0, 4)


In [17]:
len(keep_idx)

0

In [18]:
gt_dict = {"gt": dataset["answer"]}
gt_dict["gt"] = [x for i, x in enumerate(gt_dict["gt"]) if i in keep_idx]

gt_dict["question"] = [q for i,q in enumerate(dataset['question']) if i in keep_idx]

len(gt_dict["gt"])

0

In [19]:
try:
    os.mkdir(f"./data/{now}")
except:
    pass

with open(f"./data/{now}/llm_response_{n_sample}__{now}.json", "w") as json_file:
    json.dump(llm_response, json_file)

pd.DataFrame(gt_dict).to_csv(f"./data/{now}/gt_{n_sample}__{now}.csv", index=False)

In [20]:
filename = f"./data/{now}/llm_response_{n_sample}__{now}.txt"

with open(filename, "w") as file:
    for sublist in llm_result_list:
        line = " ".join(str(num) for num in sublist)  # Convert to string, join with spaces
        file.write(line + "\n")  # Write line and add newline
filename

'./data/20240813/llm_response_100__20240813.txt'

#  GLAD

In [14]:
from utils import Aggregate

In [15]:
glad = Aggregate().glad

In [23]:
glad_output = glad("./data/20240604/llm_response_10__20240604.txt")

In [24]:
glad_output

{'alpha': {0: 1.3686111104019638,
  1: 0.4609621853035853,
  2: 1.3686111104019638,
  3: 0.46096218530358546},
 'beta': {0: 2.213137577969233,
  1: 4.447469443495815,
  2: 2.213137577969233,
  3: 4.447469443495815,
  4: 4.447469443495815,
  5: 1.5354236534317327},
 'probZ': {0: {0: 0.0002598525261959247,
   1: 3.122844068982658e-05,
   2: 0.9996776905924244,
   3: 3.122844068982658e-05},
  1: {0: 1.056407524860323e-09,
   1: 1.056407524860323e-09,
   2: 0.9999999968307774,
   3: 1.056407524860323e-09},
  2: {0: 0.0002598525261959247,
   1: 0.9996776905924244,
   2: 3.122844068982658e-05,
   3: 3.122844068982658e-05},
  3: {0: 0.9999999968307772,
   1: 1.0564075248603228e-09,
   2: 1.0564075248603228e-09,
   3: 1.0564075248603228e-09},
  4: {0: 1.056407524860323e-09,
   1: 1.056407524860323e-09,
   2: 0.9999999968307774,
   3: 1.056407524860323e-09},
  5: {0: 0.0016259262187987,
   1: 0.009899281369083378,
   2: 0.9785755110430344,
   3: 0.009899281369083378}},
 'labels': {0: 2, 1: 2, 2

# Eval

In [25]:
import json
import pandas as pd
from collections import Counter
from typing import Dict, List
from sklearn.metrics import accuracy_score, confusion_matrix

In [26]:
def get_majority_vote(label_dict: Dict[str, List[int]]) -> List[int]:
    """
    Finds the majority value for each element across multiple lists within a dictionary.

    Args:
        label_dict: A dictionary where keys are identifiers and values are lists of labels.

    Returns:
        A list of majority values corresponding to each element position.
    """
    list_of_labels = list(label_dict.values())  # Extract values into a list
    majority_values = []

    for elements in zip(*list_of_labels):
        element_counts = Counter(elements)
        most_common_element = element_counts.most_common(1)[0]
        majority_values.append(most_common_element[0])

    return majority_values


def accuracy_with_none_penalty(y_true, y_pred):
    filtered_y_true = []
    filtered_y_pred = []

    for true, pred in zip(y_true, y_pred):
        if pred is not None:  # Only include non-None predictions
            filtered_y_true.append(true)
            filtered_y_pred.append(pred)
        else:
            filtered_y_true.append(true)  # Include true label
            filtered_y_pred.append(-1)   # Replace None with wrong label (e.g., -1)

    return accuracy_score(filtered_y_true, filtered_y_pred)

In [30]:
with open("./data/20240530/llm_response_2000__20240530.json", "r") as f:
    llm_response = json.load(f)

# df_glad = pd.read_csv("./data/label_glad__20240529.csv")


In [31]:
llm_response["majority"] = get_majority_vote(llm_response)
# llm_response["glad"] = df_glad["label"].values
llm_response["glad"] = list(glad_output['labels'].values())

In [32]:
d = {}
for k, v in llm_response.items():
    d[k] = accuracy_with_none_penalty(dataset['answer'], v) * 100

In [33]:
df = pd.DataFrame([d]).T.reset_index()
df.columns = ["model", "accuracy"]
df

,model,accuracy
0,gemini_-1.0-pro-001,27.000000
1,gemini_-1.5-flash-001,21.000000
2,gemini_-1.0-ultra-001,29.000000
3,palm_temp_0.4,26.000000
4,majority,25.000000
5,glad,33.333333


In [34]:
# hard questions
q_list = [q for i,q in enumerate(dataset['question']) if i in keep_idx]
glad_output["beta"]
question_d = {"question": q_list,
              "task difficulty": list(glad_output["beta"].values()),
              "label confidence": list(glad_output["probZ"].values())
}

In [36]:
llm_response.values()

dict_values([[0, 2, 1, 1, 2, 3, 2, 1, 2, 2, 1, 1, 1, 2, 1, 3, 2, 1, 2, 1, 1, 1, 2, 1, 3, 2, 1, 1, 3, 1, 0, 2, 1, 0, 3, 1, 2, 1, 1, 0, 3, 3, 2, 3, 1, 3, 0, 1, 2, 3, 3, 1, 2, 2, 3, 1, 1, 3, 2, 1, 3, 0, 2, 1, 2, 2, 0, 1, 3, 2, 2, 0, 1, 2, 0, 0, 1, 0, 2, 1, 1, 0, 3, 0, 2, 1, 1, 0, 0, 2, 3, 2, 2, 0, 0, 1, 0, 1, 1, 3, 3, 0, 3, 2, 3, 0, 3, 2, 0, 0, 1, 2, 3, 3, 3, 0, 3, 1, 3, 1, 2, 1, 0, 3, 3, 2, 1, 2, 0, 1, 2, 1, 0, 3, 1, 2, 2, 2, 1, 0, 2, 3, 1, 2, 3, 3, 2, 0, 2, 1, 3, 0, 0, 3, 3, 0, 1, 1, 1, 0, 1, 0, 1, 1, 2, 3, 0, 2, 1, 0, 0, 3, 1, 2, 3, 0, 2, 3, 3, 2, 2, 1, 2, 3, 0, 2, 3, 1, 2, 0, 2, 3, 3, 0, 1, 3, 1, 0, 1, 1, 0, 3, 3, 3, 1, 3, 0, 1, 2, 3, 0, 0, 3, 3, 0, 2, 1, 1, 2, 0, 3, 2, 1, 0, 3, 2, 3, 0, 0, 2, 2, 1, 3, 1, 3, 3, 3, 3, 0, 1, 1, 2, 1, 2, 1, 2, 3, 3, 0, 3, 2, 1, 3, 3, 3, 3, 1, 2, 0, 2, 3, 2, 0, 2, 3, 2, 1, 2, 0, 2, 1, 1, 2, 2, 3, 1, 1, 3, 2, 1, 0, 3, 2, 3, 1, 1, 2, 2, 3, 0, 2, 0, 3, 0, 3, 3, 2, 1, 1, 3, 3, 2, 3, 0, 3, 0, 1, 0, 3, 2, 2, 0, 0, 2, 3, 0, 1, 0, 3, 3, 2, 2, 0, 0, 1, 1, 2, 3, 2,

In [ ]:
import pandas as pd
from itertools import combinations

def calculate_agreement_matrix(data_dict):
    """Calculates agreement between lists in a dictionary for matching indices.

    Args:
        data_dict: A dictionary where keys are labels and values are lists of equal length.

    Returns:
        A Pandas DataFrame representing the agreement matrix.
    """

    df = pd.DataFrame(data_dict)
    keys = df.columns

    agreement_matrix = pd.DataFrame(index=keys, columns=keys)

    for key1, key2 in combinations(keys, 2):  # Iterate over all key pairs
        matches = (df[key1] == df[key2]).sum()  # Count matching values
        total = len(df)  # Total number of values
        agreement_matrix.loc[key1, key2] = agreement_matrix.loc[key2, key1] = matches / total

    return agreement_matrix

# Example Usage:
data = {'list1': [1, 1, 0, 2, 1],
        'list2': [1, 0, 0, 2, 1],
        'list3': [1, 1, 1, 0, 1]}

agreement_matrix = calculate_agreement_matrix(data)
print(agreement_matrix.to_markdown(numalign="left", stralign="left"))


In [ ]:
dm = calculate_agreement_matrix(llm_response)
dm